<h1 style="text-align: center;">Pre-processing the real estate transaction records</h1>

<em><strong>Note:</strong><br>
This notebook is part of a project analysing France’s territory and the house prices. Please read the <a href=" https://github.com/Ashish-3/House-prices-in-France/blob/master/Readme.md">readme</a> file for more information:</em>



In this notebook we will pre-process the real estate transaction, and extract information about house prices in France. This will data later be used during the data analysis step.

## Importing libraries

In [2]:
import numpy as np
import pandas as pd

#show all the columns and rows of a data
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import seaborn as sns

import time

## Importing data 

In [56]:
# Import CSV data
val_immo= pd.read_csv(r'data/valeursfoncieres-2019.txt', sep="|", dtype=str)
val_immo= val_immo.append(pd.read_csv(r'data/valeursfoncieres-2018.txt', sep="|", dtype=str))
val_immo= val_immo.append(pd.read_csv(r'data/valeursfoncieres-2017.txt', sep="|", dtype=str))
val_immo= val_immo.append(pd.read_csv(r'data/valeursfoncieres-2016.txt', sep="|", dtype=str))
val_immo= val_immo.append(pd.read_csv(r'data/valeursfoncieres-2015.txt', sep="|", dtype=str))

print('Data succesfuly loaded!')

Data succesfuly loaded!


In [3]:
print(val_immo.shape)
val_immo.head()

(14880473, 43)


,Code service CH,Reference document,1 Articles CGI,2 Articles CGI,3 Articles CGI,4 Articles CGI,5 Articles CGI,No disposition,Date mutation,Nature mutation,Valeur fonciere,No voie,B/T/Q,Type de voie,Code voie,Voie,Code postal,Commune,Code departement,Code commune,Prefixe de section,Section,No plan,No Volume,1er lot,Surface Carrez du 1er lot,2eme lot,Surface Carrez du 2eme lot,3eme lot,Surface Carrez du 3eme lot,4eme lot,Surface Carrez du 4eme lot,5eme lot,Surface Carrez du 5eme lot,Nombre de lots,Code type local,Type local,Identifiant local,Surface reelle bati,Nombre pieces principales,Nature culture,Nature culture speciale,Surface terrain
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,000001,04/01/2019,Vente,"37220,00",26,NaN,RUE,2730,DE MONTHOLON,1000,BOURG-EN-BRESSE,01,53,NaN,AI,298,NaN,8,"19,27",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2,Appartement,NaN,20,1,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,000001,04/01/2019,Vente,"185100,00",22,NaN,RUE,1650,GEN DELESTRAINT,1000,BOURG-EN-BRESSE,01,53,NaN,AM,95,NaN,137,NaN,154,"61,51",NaN,NaN,NaN,NaN,NaN,NaN,2,2,Appartement,NaN,62,3,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,000001,04/01/2019,Vente,"185100,00",22,B,RUE,1650,GEN DELESTRAINT,1000,BOURG-EN-BRESSE,01,53,NaN,AM,95,NaN,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,3,Dépendance,NaN,0,0,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,000001,08/01/2019,Vente,"209000,00",3,NaN,RUE,0043,DES CHAMPAGNES,1160,PRIAY,01,314,NaN,E,1676,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,Maison,NaN,90,4,S,NaN,940
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,000001,07/01/2019,Vente,"134900,00",5,NaN,LOT,A003,LE BIOLAY,1370,SAINT-ETIENNE-DU-BOIS,01,350,NaN,AA,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,Maison,NaN,101,5,S,NaN,490


The DataFrame has 14880473 rows in total

## Data pre-processing

###### Preliminairy data cleanning

In [57]:
start = time.time()

# Dropping empty columns
val_immo.drop(['Code service CH', 'Reference document', '1 Articles CGI', '2 Articles CGI', '3 Articles CGI', '4 Articles CGI','5 Articles CGI', 'Identifiant local'], axis=1, inplace=True)

# Dropping other unuseful columns
val_immo.drop(['No disposition', 'B/T/Q', 'Prefixe de section', 'Section', 'No plan', 'No Volume','1er lot','2eme lot','3eme lot','4eme lot','5eme lot','Date mutation', 'No voie', 'Type de voie' ,'Code voie', 'Nombre de lots', 'Code commune'], axis=1, inplace=True)

# Keeping only the transactions that concerns sales ("vente" in french)
# it represent more than 90% of the dataset
# Dropping the 'Nature mutation' columns after filtering the sales
val_immo=val_immo[val_immo['Nature mutation']=='Vente']
val_immo.drop(['Nature mutation'], axis=1, inplace=True)

# Drop the transaction  where the type of real estate column is left empty
val_immo.dropna(subset=["Code type local"], axis=0, inplace=True)

# Converting "Nombre pieces principales" (amount of rooms) into integer
val_immo.loc[:,'Nombre pieces principales']=val_immo.loc[:,'Nombre pieces principales'].astype(float)

# Converting "Valeur fonciere" (real estate value) into float
val_immo.loc[:,'Valeur fonciere']=val_immo.loc[:,'Valeur fonciere'].replace(",",".", regex=True)
val_immo.loc[:,'Valeur fonciere']=val_immo.loc[:,'Valeur fonciere'].astype(float)

# Converting "Surface terrain" (land surface) into float
val_immo.loc[:,'Surface terrain']=val_immo.loc[:,'Surface terrain'].replace(",",".", regex=True)
val_immo.loc[:,'Surface terrain']=val_immo.loc[:,'Surface terrain'].astype(float)

# Reset the index of the dataset
val_immo.reset_index(drop=True, inplace=True)


end = time.time()
temps=end - start
total=14900000
print('Temps de traitement :',temps, 'secondes pour ', val_immo.shape[0], 'lignes')
print('Temps nécessaire pour traiter', total, 'lignes :', (temps*total)/(val_immo.shape[0]*60*60) ,'heures')

print(val_immo.shape)
val_immo.head(2)

Temps de traitement : 0.06196451187133789 secondes pour  72 lignes
Temps nécessaire pour traiter 14900000 lignes : 3.5620031901347784 heures
(72, 17)


,Valeur fonciere,Voie,Code postal,Commune,Code departement,Surface Carrez du 1er lot,Surface Carrez du 2eme lot,Surface Carrez du 3eme lot,Surface Carrez du 4eme lot,Surface Carrez du 5eme lot,Code type local,Type local,Surface reelle bati,Nombre pieces principales,Nature culture,Nature culture speciale,Surface terrain
0,37220.0,DE MONTHOLON,1000,BOURG-EN-BRESSE,01,"19,27",NaN,NaN,NaN,NaN,2,Appartement,20,1.0,NaN,NaN,NaN
1,185100.0,GEN DELESTRAINT,1000,BOURG-EN-BRESSE,01,NaN,"61,51",NaN,NaN,NaN,2,Appartement,62,3.0,NaN,NaN,NaN


######  Getting the surfaces of the transactions right

In [5]:
start = time.time()

# Summarizing the total of "Surface Carrez" (living area) into one column and dropping the others
# Using numpy for faster processing

surface_carrez=np.char.replace(val_immo["Surface Carrez du 1er lot"].to_numpy().astype(np.str),",",".").astype(float)
+np.char.replace(val_immo["Surface Carrez du 2eme lot"].to_numpy().astype(np.str),",",".").astype(float)
+np.char.replace(val_immo["Surface Carrez du 3eme lot"].to_numpy().astype(np.str),",",".").astype(float)
+np.char.replace(val_immo["Surface Carrez du 4eme lot"].to_numpy().astype(np.str),",",".").astype(float)
+np.char.replace(val_immo["Surface Carrez du 5eme lot"].to_numpy().astype(np.str),",",".").astype(float)



# Sometime the "surface Carrez"(living area) is empty
# We can replace the empty fields by the "Surface reel bati" (building surface area)
surface_bati= np.char.replace(val_immo["Surface reelle bati"].to_numpy().astype(np.str),",",".").astype(float)
for i in range(0,surface_carrez.shape[0]) :
    if surface_carrez[i]==0 or np.isnan(surface_carrez[i]):
        surface_carrez[i]=surface_bati[i]
val_immo["Surface Carrez"]=surface_carrez


# Drop the columns used to create the column 'Surface Carrez'
val_immo.drop([ 'Surface Carrez du 1er lot','Surface Carrez du 2eme lot','Surface Carrez du 3eme lot','Surface Carrez du 4eme lot','Surface Carrez du 5eme lot'], axis=1, inplace=True)
val_immo.drop([ 'Surface reelle bati'], axis=1, inplace=True)

end = time.time()
temps=end - start
total=14900000
print('Temps de traitement :',temps, 'secondes pour ', val_immo.shape[0], 'lignes')
print('Temps nécessaire pour traiter', total, 'lignes :', (temps*total)/(val_immo.shape[0]*60*60) ,'heures')

print(val_immo.shape)
val_immo.head(3)

Temps de traitement : 120.93264579772949 secondes pour  7549422 lignes
Temps nécessaire pour traiter 14900000 lignes : 0.06630001396082484 heures
(7549422, 13)


,Valeur fonciere,Voie,Code postal,Commune,Code departement,Code commune,Code type local,Type local,Nombre pieces principales,Nature culture,Nature culture speciale,Surface terrain,Surface Carrez
0,37220.0,DE MONTHOLON,1000,BOURG-EN-BRESSE,01,53,2,Appartement,1.0,NaN,NaN,NaN,19.27
1,185100.0,GEN DELESTRAINT,1000,BOURG-EN-BRESSE,01,53,2,Appartement,3.0,NaN,NaN,NaN,62.00
2,185100.0,GEN DELESTRAINT,1000,BOURG-EN-BRESSE,01,53,3,Dépendance,0.0,NaN,NaN,NaN,0.00


######  Removing duplicates

Let's remove : 
     - transactions with outbuildings ("Dépendance") 
     - transactions that may have been recorded twice
     - or mutliple buildings that were recorded in seperate transactions but were bought together (in bulk purchase)
They all share the same characteristics : 
      - they have two rows with the excat same transaction price ('Valeur fonciere') next to each other
 Let's tag those transactions and remove them from the dataset.
 For faster computing we will use numpy insted of pandas

In [6]:
# inatializing the variables with 'marker' the np array used to tag the rows to be removed
fonciere=val_immo['Valeur fonciere'].to_numpy()
marker=np.full((fonciere.shape[0]), False)

# using a loop to tag the transactions
counter = 0
start = time.time()
for i in range (0,fonciere.shape[0]-1) :
    if fonciere[i]==fonciere[i+1]:
        marker[1]=True
        marker[i+1]=True
        counter = counter + 1
end = time.time()
temps=end - start
total=14900000
print('Temps de traitement :',temps, 'secondes', 'pour', fonciere.shape[0], 'lignes')
print('Nombre de duplicatat trouvé :', counter)
print('Temps nécessaire pour traiter', total, 'lignes :', (temps*total)/(val_immo.shape[0]*60*60) ,'heures')

# dropping the transactions
dropped=val_immo.shape[0]
val_immo.drop(np.where(marker==True)[0], axis=0, inplace=True)
dropped=dropped-val_immo.shape[0]
print(dropped, 'lines dropped')

# Reset the index of the dataset after dropping some rows
val_immo.reset_index(drop=True, inplace=True)

print(val_immo.shape)
val_immo.head(3)

Temps de traitement : 5.526273727416992 secondes pour 7549422 lignes
Nombre de duplicatat trouvé : 3000698
Temps nécessaire pour traiter 14900000 lignes : 0.0030297197490569178 heures
3000699 lines dropped
(4548723, 13)


,Valeur fonciere,Voie,Code postal,Commune,Code departement,Code commune,Code type local,Type local,Nombre pieces principales,Nature culture,Nature culture speciale,Surface terrain,Surface Carrez
0,37220.0,DE MONTHOLON,1000,BOURG-EN-BRESSE,01,53,2,Appartement,1.0,NaN,NaN,NaN,19.27
1,209000.0,DES CHAMPAGNES,1160,PRIAY,01,314,1,Maison,4.0,S,NaN,940.0,90.00
2,134900.0,LE BIOLAY,1370,SAINT-ETIENNE-DU-BOIS,01,350,1,Maison,5.0,S,NaN,490.0,101.00


###### For this sutdy : let's study only houses transactions

Let's see what are the proportions of houses (maison) and flats (appartement) do we have in the DataFrame

In [7]:
print('\n','Pourcentage maison',val_immo[val_immo['Code type local']=='1'].shape[0]/val_immo.shape[0],
      '\n',
      'Pourcentage appartement',val_immo[val_immo['Code type local']=='2'].shape[0]/val_immo.shape[0])


 Pourcentage maison 0.4858381132462891 
 Pourcentage appartement 0.29528001595172976


In [8]:
# We will focus our study only on houses, therfor we will drop all  other transaction type
val_immo = val_immo[ val_immo['Code type local']=='1' ]
val_immo.reset_index(drop=True, inplace=True)

print(val_immo.shape)
val_immo.head(3)

(2209943, 13)


,Valeur fonciere,Voie,Code postal,Commune,Code departement,Code commune,Code type local,Type local,Nombre pieces principales,Nature culture,Nature culture speciale,Surface terrain,Surface Carrez
0,209000.0,DES CHAMPAGNES,1160,PRIAY,01,314,1,Maison,4.0,S,NaN,940.0,90.0
1,134900.0,LE BIOLAY,1370,SAINT-ETIENNE-DU-BOIS,01,350,1,Maison,5.0,S,NaN,490.0,101.0
2,192000.0,DES LIBELLULES,1340,ATTIGNAT,01,24,1,Maison,4.0,S,NaN,708.0,88.0


In [9]:
# Let's add a price per meter square column and get a quick description of the DataFrame
print(val_immo.shape)
val_immo["Prix_m2"]=val_immo['Valeur fonciere']/val_immo['Surface Carrez']
val_immo.describe(include='all')

(2209943, 13)


,Valeur fonciere,Voie,Code postal,Commune,Code departement,Code commune,Code type local,Type local,Nombre pieces principales,Nature culture,Nature culture speciale,Surface terrain,Surface Carrez,Prix_m2
count,2.200766e+06,2209161,2209003,2209196,2209943,2209943,2209943,2209943,2.209914e+06,2106595,41088,2.106595e+06,2.209914e+06,2.200737e+06
unique,NaN,383945,5863,33168,97,908,1,1,NaN,27,100,NaN,NaN,NaN
top,NaN,GRANDE RUE,59200,NANTES,59,63,1,Maison,NaN,S,POTAG,NaN,NaN,NaN
freq,NaN,13046,4438,6303,113686,13460,2209943,2209943,NaN,1879834,18317,NaN,NaN,NaN
mean,2.162659e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.209123e+00,NaN,NaN,7.200137e+02,1.007709e+02,2.229404e+03
std,1.965079e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.468628e+00,NaN,NaN,3.681187e+03,4.570260e+01,1.841632e+04
min,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,NaN,NaN,0.000000e+00,9.200000e-01,0.000000e+00
25%,1.059900e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.000000e+00,NaN,NaN,2.600000e+02,7.500000e+01,1.187500e+03
50%,1.680000e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.000000e+00,NaN,NaN,5.000000e+02,9.300000e+01,1.777778e+03
75%,2.550000e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.000000e+00,NaN,NaN,7.800000e+02,1.200000e+02,2.607407e+03


Let's have a quick idea of how big are most of the house :

In [11]:
print('The "Surface terrain" where we reach the 85th quantile is :',
      val_immo['Surface terrain'].quantile(q=0.85),'m2',
      '\n',
      'The "Surface Carrez" where we reach the 95th quantile is :',
      val_immo['Surface Carrez'].quantile(q=0.97),'m2',
      '\n',
     'The "Nombre pieces principales" where we reach the 99,95th quantile is :',
      val_immo['Nombre pieces principales'].quantile(q=0.9995),'m2')

The "Surface terrain" where we reach the 85th quantile is : 1000.0 m2 
 The "Surface Carrez" where we reach the 95th quantile is : 198.0 m2 
 The "Nombre pieces principales" where we reach the 99,95th quantile is : 13.0 m2


Therefor, for this study let's remove the outliers by removing houses with :
    - a living area bigger than 200 m2 
    - a land surface bigger than 1000 m2
    - more than 10 rooms

In [12]:
# Slicing the data
val_immo=val_immo[ (val_immo['Surface Carrez']<=200) &
                  (val_immo['Surface terrain']<=1000) &
                 (val_immo['Nombre pieces principales']<=10)]

# Reset the index of the dataset
val_immo.reset_index(drop=True, inplace=True)

val_immo.shape

(1775051, 14)

In [3]:
#val_immo.to_csv( r'val_immo.csv', index=False)
val_immo=pd.read_csv( r'val_immo.csv')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


sns.distplot(val_immo['Surface Carrez'] )

sns.regplot(x='Surface terrain', 
            y='Valeur fonciere', 
            data=val_immo.head(10000)[val_immo.head(10000)['Valeur fonciere']<300000], 
            color='green', 
            scatter_kws={'s': 1})

## Summarising the house sales per Communes

Let's summarise the transaction per Communes in a DataFrame called com_immo

In [42]:
# Calculating the mean values per Commune
com_immo=val_immo.groupby('Commune').mean()


# Storing the number of observations per Commune in a column called 'N'
com_immo['N']=val_immo.groupby('Commune').size()

com_immo.reset_index(inplace=True)
com_immo.head()

,Commune,Valeur fonciere,Code postal,Code commune,Code type local,Nombre pieces principales,Surface terrain,Surface Carrez,Prix_m2,N
0,AAST,129665.200000,64460.000000,1.0,1.0,4.600000,608.400000,118.600000,1075.662291,5
1,ABAINVILLE,63735.164286,55130.000000,1.0,1.0,3.857143,396.142857,88.285714,682.972083,14
2,ABANCOURT,120205.960952,59902.666667,1.0,1.0,4.095238,553.095238,90.904762,1379.239892,42
3,ABAUCOURT,133970.214286,54610.000000,1.0,1.0,3.857143,441.571429,109.071429,1360.566532,14
4,ABAUCOURT-HAUTECOURT,84500.000000,55400.000000,2.0,1.0,6.000000,696.000000,142.000000,609.778226,2


###  Let's try to merge our house prices (com_immo) to other info about Communes (all_data)

We load the data have about age, revenues and geography  for the Communes

In [51]:
all_data=pd.read_csv(r'all_data.csv')
all_data.head()

,Code INSEE,Code Postal,Commune,Code Département,Code Région,Superficie_km2,lat,lng,age_mean,age_std,age_median,population,merge1,Fhousehold,person_p_Fhousehold,revenue_median,revenue_inequalities,merge2,density
0,59276,59287,GUESNAIN,59,31.0,4.05,50.348346,3.148167,40.851860,24.642605,41.0,4651.0,both,1884.0,4624.0,17270.0,2.6,both,1148.395062
1,88128,88210,DENIPAIRE,88,41.0,7.02,48.339817,6.961893,43.832158,22.810009,48.0,246.0,both,108.0,246.0,21720.0,NaN,both,35.042735
2,57538,57170,PETTONCOURT,57,41.0,4.93,48.788126,6.411069,40.315972,23.739636,41.0,288.0,both,109.0,298.0,20970.0,NaN,both,58.417850
3,54459,54630,RICHARDMENIL,54,41.0,7.07,48.595865,6.176145,46.335881,23.049189,51.0,2358.0,both,1020.0,2395.0,25960.0,2.4,both,333.521924
4,35022,35190,BECHEREL,35,53.0,0.55,48.296509,-1.942824,41.722078,25.144570,42.0,673.0,both,293.0,632.0,19260.0,NaN,both,1223.636364


####  First try to merge the data

 As the the real estate data doesn't have any columns stating the Code INSEE of the Communes, the only way of merge the data is by using Commune names as the merging key.

In [18]:
merge=pd.merge(all_data,com_immo,on='Commune', how='outer', indicator=True)

In [20]:
merge.head()

,Code INSEE,Code Postal,Commune,Code Commune,Superficie_km2,lat,lng,age_mean,age_std,age_median,population,merge1,Fhousehold,person_p_Fhousehold,revenue_median,revenue_inequalities,merge2,density,Valeur fonciere,Nombre pieces principales,Surface terrain,Surface Carrez,Prix_m2,N,_merge
0,59276,59287,GUESNAIN,276.0,4.05,50.348346,3.148167,40.851860,24.642605,41.0,4651.0,both,1884.0,4624.0,17270.0,2.6,both,1148.395062,126111.978261,4.391304,410.869565,93.391304,1368.677765,NaN,both
1,88128,88210,DENIPAIRE,128.0,7.02,48.339817,6.961893,43.832158,22.810009,48.0,246.0,both,108.0,246.0,21720.0,NaN,both,35.042735,98260.000000,4.400000,456.400000,99.200000,1002.946560,NaN,both
2,57538,57170,PETTONCOURT,538.0,4.93,48.788126,6.411069,40.315972,23.739636,41.0,288.0,both,109.0,298.0,20970.0,NaN,both,58.417850,NaN,NaN,NaN,NaN,NaN,NaN,left_only
3,54459,54630,RICHARDMENIL,459.0,7.07,48.595865,6.176145,46.335881,23.049189,51.0,2358.0,both,1020.0,2395.0,25960.0,2.4,both,333.521924,201593.181818,4.738636,493.113636,107.909091,1924.344829,NaN,both
4,35022,35190,BECHEREL,22.0,0.55,48.296509,-1.942824,41.722078,25.144570,42.0,673.0,both,293.0,632.0,19260.0,NaN,both,1223.636364,142893.235294,4.823529,327.500000,117.735294,1239.205157,NaN,both


In [19]:
print('\n','Rows in all_data', all_data.shape[0], '\n',
      'Rows in com_immo', com_immo.shape[0], '\n',
    'Rows successfuly merged :', merge[merge['_merge']=='both'].shape[0],
      '\n',
    'Rows in all_data but not in com_immo :', merge[merge['_merge']=='left_only'].shape[0],
      '\n',
    'Rows in com_immo but not in all_data :', merge[merge['_merge']=='right_only'].shape[0])


 Rows in all_data 36748 
 Rows in com_immo 32700 
 Rows successfuly merged : 29143 
 Rows in all_data but not in com_immo : 7605 
 Rows in com_immo but not in all_data : 6138


We couldn't merge around 6100 rows of com_immo into the dataframe all_data (which is supposedly more comprehensive). 
<br><br>
Many rows could not merge because they do not have the exact same way of spelling/writing the Commune name.
For instance the Commune 'ABBANS-DESSOUS' is written like that 'ABBANS DESSOUS' in com_immo, or the commune 'L'ABERGEMENT-SAINTE-COLOMBE' is written 'ABERGEMENT STE COLOMBE L'. <br> Let's try to correct the Commune names of the rows that couldn't be merged.

#### Correcting the name of Communes in com_immo before trying to merge the data again

To correct the names of the Commune first we will create 2 lists :
- a list called list_inaccurate_label to gather all Communes names from the rows of com_immo that couldn't be merged. 
- a list called list_of_choices to gather all Communes names from the rows of all_data that couldn't be matched. 

In [23]:
list_inaccurate_label=list(merge.Commune[merge['_merge']=='right_only'])
list_of_choices=list(merge.Commune[merge['_merge']=='left_only'])
print('\n','Size of list_inaccurate_label' , len(list_inaccurate_label),'\n', 'Size of list_of_choices', len(list_of_choices))
print(list_inaccurate_label[0:5])
print(list_of_choices[0:5])


 Size of list_inaccurate_label 6138 
 Size of list_of_choices 7605
['ABBANS DESSOUS', 'ABBANS DESSUS', 'ABBEVILLE ST LUCIEN', 'ABERGEMENT STE COLOMBE L', 'ABERGEMENT-DE-CUISERY (L )']
['PETTONCOURT', 'SAINTE-MARIE-DE-VATIMESNIL', "COLLONGES-AU-MONT-D'OR", 'SICKERT', "LES ROCHES-L'EVEQUE"]


We will use a library called fuzzywuzzy that will allow us to match: the names of Communes in com_immo, to the came of Commune in  all_data. This library will return a score indicating how similar they names are. 

In [24]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

Creating a dataframe called correction that shows the inaccurate and accurate labels of Communes next to their matching score 

In [25]:
start = time.time()

correction=[]
for innaccurate_label in list_inaccurate_label :
    output=process.extractOne(innaccurate_label, list_of_choices, scorer=fuzz.token_sort_ratio)
    correction.append([output[1], innaccurate_label, output[0]])
correction
correction=pd.DataFrame(correction, columns=['score', 'inaccurate_label', 'corrected_label'])

end = time.time()
temps=end - start
print('Temps de traitement :',temps, 'secondes')

Temps de traitement : 652.6506822109222 secondes


In [43]:
#correction.to_csv(r'correction.csv', index=False)
correction=pd.read_csv(r'correction.csv')
print(correction.shape)
correction.head()

(6138, 3)


,score,inaccurate_label,corrected_label
0,100,ABBANS DESSOUS,ABBANS-DESSOUS
1,100,ABBANS DESSUS,ABBANS-DESSUS
2,93,ABBEVILLE ST LUCIEN,ABBEVILLE-SAINT-LUCIEN
3,94,ABERGEMENT STE COLOMBE L,L'ABERGEMENT-SAINTE-COLOMBE
4,100,ABERGEMENT-DE-CUISERY (L ),L'ABERGEMENT-DE-CUISERY


#### Manually correcting the name of some boroughs

The name of the boroughs in the cities of Lyon, Marseille and Paris are too different and can't be matched with fuzzywuzzy, we will match them manually. To do so, I have created a csv file calle cities_boroughs with the accurrate labeling of the boroughs.

In [44]:
# Load the manually matched labels of the boroughs
cities_boroughs=pd.read_csv(r'cities_boroughs.csv')
print(cities_boroughs.shape)
cities_boroughs.head(3)

(45, 3)


,score,inaccurate_label,corrected_label
0,100,PARIS 01,PARIS-1ER-ARRONDISSEMENT
1,100,PARIS 02,PARIS-2E-ARRONDISSEMENT
2,100,PARIS 03,PARIS-3E-ARRONDISSEMENT


In [45]:
# Merging the loaded labels
correction=correction.append(cities_boroughs)
print(correction.shape)

(6183, 3)


In [46]:
# Keeping only matched labels with an score above 92
correction=correction[correction.score>92].sort_values('score', )
correction.reset_index(drop=True, inplace=True)
print(correction.shape)
correction.head(3)

(4608, 3)


,score,inaccurate_label,corrected_label
0,93,ST GERMAIN LA POTERIE,SAINT-GERMAIN-LA-POTERIE
1,93,ST MARTIN DE GURSON,SAINT-MARTIN-DE-GURSON
2,93,CHANTENAY-ST-IMBERT,CHANTENAY-SAINT-IMBERT


#### Applying the correction to com_immo

In [47]:
#corrected_com_immo=com_immo.copy()
for i in range(0,correction.shape[0]) :
    com_immo.loc[com_immo.Commune==correction.inaccurate_label[i],'Commune']=correction.corrected_label[i]      
print(com_immo.shape)
com_immo.head()

(32700, 10)


,Commune,Valeur fonciere,Code postal,Code commune,Code type local,Nombre pieces principales,Surface terrain,Surface Carrez,Prix_m2,N
0,AAST,129665.200000,64460.000000,1.0,1.0,4.600000,608.400000,118.600000,1075.662291,5
1,ABAINVILLE,63735.164286,55130.000000,1.0,1.0,3.857143,396.142857,88.285714,682.972083,14
2,ABANCOURT,120205.960952,59902.666667,1.0,1.0,4.095238,553.095238,90.904762,1379.239892,42
3,ABAUCOURT,133970.214286,54610.000000,1.0,1.0,3.857143,441.571429,109.071429,1360.566532,14
4,ABAUCOURT-HAUTECOURT,84500.000000,55400.000000,2.0,1.0,6.000000,696.000000,142.000000,609.778226,2


In [49]:
#com_immo.to_csv(r'com_immo.csv', index=False)
com_immo=pd.read_csv(r'com_immo.csv')
com_immo.head()

,Commune,Valeur fonciere,Code postal,Code commune,Code type local,Nombre pieces principales,Surface terrain,Surface Carrez,Prix_m2,N
0,AAST,129665.200000,64460.000000,1.0,1.0,4.600000,608.400000,118.600000,1075.662291,5
1,ABAINVILLE,63735.164286,55130.000000,1.0,1.0,3.857143,396.142857,88.285714,682.972083,14
2,ABANCOURT,120205.960952,59902.666667,1.0,1.0,4.095238,553.095238,90.904762,1379.239892,42
3,ABAUCOURT,133970.214286,54610.000000,1.0,1.0,3.857143,441.571429,109.071429,1360.566532,14
4,ABAUCOURT-HAUTECOURT,84500.000000,55400.000000,2.0,1.0,6.000000,696.000000,142.000000,609.778226,2


#### Merging the data

In [52]:
all_data_immo=pd.merge(all_data,com_immo,on='Commune', how='outer', indicator=True)
all_data_immo.reset_index(drop=True, inplace=True)

In [55]:
#all_data_immo.to_csv(r'all_data_immo.csv', index=False)
#all_data_immo=pd.read_csv(r'all_data_immo.csv')
all_data_immo.head()

,Code INSEE,Code Postal,Commune,Code Département,Code Région,Superficie_km2,lat,lng,age_mean,age_std,age_median,population,merge1,Fhousehold,person_p_Fhousehold,revenue_median,revenue_inequalities,merge2,density,Valeur fonciere,Code postal,Code commune,Code type local,Nombre pieces principales,Surface terrain,Surface Carrez,Prix_m2,N,_merge
0,59276,59287,GUESNAIN,59,31.0,4.05,50.348346,3.148167,40.851860,24.642605,41.0,4651.0,both,1884.0,4624.0,17270.0,2.6,both,1148.395062,126111.978261,59287.0,276.0,1.0,4.391304,410.869565,93.391304,1368.677765,138.0,both
1,88128,88210,DENIPAIRE,88,41.0,7.02,48.339817,6.961893,43.832158,22.810009,48.0,246.0,both,108.0,246.0,21720.0,NaN,both,35.042735,98260.000000,88210.0,128.0,1.0,4.400000,456.400000,99.200000,1002.946560,5.0,both
2,57538,57170,PETTONCOURT,57,41.0,4.93,48.788126,6.411069,40.315972,23.739636,41.0,288.0,both,109.0,298.0,20970.0,NaN,both,58.417850,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
3,54459,54630,RICHARDMENIL,54,41.0,7.07,48.595865,6.176145,46.335881,23.049189,51.0,2358.0,both,1020.0,2395.0,25960.0,2.4,both,333.521924,201593.181818,54630.0,459.0,1.0,4.738636,493.113636,107.909091,1924.344829,88.0,both
4,35022,35190,BECHEREL,35,53.0,0.55,48.296509,-1.942824,41.722078,25.144570,42.0,673.0,both,293.0,632.0,19260.0,NaN,both,1223.636364,142893.235294,35190.0,22.0,1.0,4.823529,327.500000,117.735294,1239.205157,34.0,both


Let's analyse how the merging process went :

In [54]:
print('\n','Rows in all_data', all_data.shape[0], '\n',
      'Rows in corrected_com_immo', com_immo.shape[0], '\n',
    'Rows successfuly merged :', all_data_immo[all_data_immo['_merge']=='both'].shape[0],
      '\n',
    'Rows in all_data but not in corrected_com_immo :', all_data_immo[all_data_immo['_merge']=='left_only'].shape[0],
      '\n',
    'Rows in com_immo but not in all_data :', all_data_immo[all_data_immo['_merge']=='right_only'].shape[0])


 Rows in all_data 36748 
 Rows in corrected_com_immo 32700 
 Rows successfuly merged : 33770 
 Rows in all_data but not in corrected_com_immo : 2999 
 Rows in com_immo but not in all_data : 1536


We managed to merge most of the data. Compared to the first try merging the data, we merge around 4500 rows more. Unfortunately, around 1500 rows over a total of 32500 still couldn't be merge. There is room for improvment here, but some of the communes were actually modified by the French government along the years. We will have to ignore that in this case study. 